In [2]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from pyro import clear_param_store
import pyro.contrib.gp as gp
from pyro.nn import PyroSample
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS, Predictive,HMC
import torch
import arviz as az
from scipy.optimize import curve_fit

In [3]:
train=pd.read_excel(r"../data/chem_train_pm.xlsx")

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
x=torch.tensor(train.corrected_week).float().to(device)
y_pm=torch.tensor(train.pm25_Gaussion).float().to(device)
y_bc=torch.tensor(train.BC_Gaussion).float().to(device)

## PM2.5

In [6]:
clear_param_store()
rbf = gp.kernels.RBF(input_dim=1)


rbf.variance = PyroSample(dist.HalfNormal(torch.tensor(train.pm25_Gaussion.mean())))

rbf.lengthscale = PyroSample(dist.HalfNormal(torch.tensor(20.)))
gpr = gp.models.GPRegression(x,y_pm, rbf).to(device)
gpr.noise = PyroSample(dist.HalfNormal(torch.tensor(40.)))


nuts_kernel = NUTS(gpr.model)


mcmc = MCMC(nuts_kernel,warmup_steps=8000, num_samples=1000,num_chains=1)

mcmc.run()

Sample: 100%|████████████████████████████████████████| 9000/9000 [03:00, 49.85it/s, step size=7.05e-01, acc. prob=0.908]


In [7]:
torch.save(gpr, "../models/pm25_seasonality_synthertic");

In [8]:
posterior_samples = mcmc.get_samples(500)
posterior_predictive= Predictive(gpr, posterior_samples)(x)
prior = Predictive(gpr, num_samples=500)(x)

pyro_data = az.from_pyro(mcmc,
    prior=prior,
    posterior_predictive=posterior_predictive,

)
az.to_json(pyro_data, "../Arviz_stats/mcmc_pm25_synthertic.json")

/data/michaelf/miniconda3/envs/gaussion_process/lib/python3.11/site-packages/arviz/data/io_pyro.py:158: UserWarning: Could not get vectorized trace, log_likelihood group will be omitted. Check your model vectorization or set log_likelihood=False
  warnings.warn(


'../Arviz_stats/mcmc_pm25_synthertic.json'

In [9]:
clear_param_store()
rbf_bc = gp.kernels.RBF(input_dim=1)


rbf_bc.variance = PyroSample(dist.HalfNormal(torch.tensor(train.BC_Gaussion.mean())))

rbf_bc.lengthscale = PyroSample(dist.HalfNormal(torch.tensor(20.)))
gpr_bc = gp.models.GPRegression(x,y_bc, rbf_bc).to(device)
gpr_bc.noise = PyroSample(dist.HalfNormal(torch.tensor(3.)))


nuts_kernel_bc = NUTS(gpr_bc.model)


mcmc_bc = MCMC(nuts_kernel_bc,warmup_steps=9000, num_samples=1000,num_chains=1)

mcmc_bc.run()

Sample: 100%|██████████████████████████████████████| 10000/10000 [03:45, 44.41it/s, step size=5.13e-01, acc. prob=0.950]


In [10]:
torch.save(gpr_bc, "../models/seasonality_BC_synthertic");

In [11]:
posterior_samples_bc = mcmc_bc.get_samples(500)
posterior_predictive_bc = Predictive(gpr_bc, posterior_samples_bc)(x)
prior_bc = Predictive(gpr_bc, num_samples=500)(x)

pyro_data_bc = az.from_pyro(mcmc_bc,
    prior=prior_bc,
    posterior_predictive=posterior_predictive_bc,

)
az.to_json(pyro_data_bc, "../Arviz_stats/mcmc_BC_synthertic.json")

'../Arviz_stats/mcmc_BC_synthertic.json'

In [12]:
az.summary(pyro_data_bc),az.summary(pyro_data)

arviz - WARNING - Shape validation failed: input_shape: (1, 1000), minimum_shape: (chains=2, draws=4)
arviz - WARNING - Shape validation failed: input_shape: (1, 1000), minimum_shape: (chains=2, draws=4)


(                      mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  \
 kernel.lengthscale  22.269  3.597  15.491   28.565      0.151    0.117   
 kernel.variance      0.865  0.431   0.198    1.661      0.016    0.015   
 noise                0.284  0.012   0.262    0.307      0.000    0.000   
 
                     ess_bulk  ess_tail  r_hat  
 kernel.lengthscale     589.0     463.0    NaN  
 kernel.variance        781.0     692.0    NaN  
 noise                  972.0     551.0    NaN  ,
                        mean      sd   hdi_3%  hdi_97%  mcse_mean  mcse_sd  \
 kernel.lengthscale   20.089   4.130   11.881   27.712      0.178    0.146   
 kernel.variance      57.090  11.243   36.084   78.123      0.377    0.369   
 noise               266.493  10.671  247.798  286.954      0.362    0.310   
 
                     ess_bulk  ess_tail  r_hat  
 kernel.lengthscale     577.0     541.0    NaN  
 kernel.variance        884.0     760.0    NaN  
 noise                  871.0     678.0 